In [1]:
import re
from pathlib import Path
import itertools

In [2]:
test_input = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
"""

In [3]:
SENSOR_PATTERN = re.compile("Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")

class Sensor:
    def __init__(self, sensor, beacon):
        self.sensor = sensor
        self.beacon = beacon
        self.distance = manhattan(sensor, beacon)
        self.min_y = int(self.sensor.imag - self.distance)
        self.max_y = int(self.sensor.imag + self.distance)
    
    def on_row(self, row):
        return self.min_y <= row <= self.max_y
            
    def skip_on_row(self, row):
        offset = int(abs(row - self.sensor.imag))
        zone_width = 2*(self.distance - offset) + 1
        zone_start = int(self.sensor.real - zone_width // 2)
        return zone_start, zone_start + zone_width
        
    def impossible_in_row(self, row):
        offset = int(abs(row - self.sensor.imag))
        zone_width = 2*(self.distance - offset) + 1
        zone_start = int(self.sensor.real - zone_width // 2)
        #print(f"{self.sensor=} {row=} {self.distance=} {zone_width=} {zone_start=}")
        return {complex(x, row) for x in range(zone_start, zone_start + zone_width) if complex(x, row) != self.beacon}

    def sensor_position(self):
        return int(self.sensor.real), int(self.sensor.imag)
    
    def beacon_position(self):
        return int(self.beacon.real), int(self.beacon.imag)


def parse_input(sensor_input):
    return [
        Sensor(complex(a, b), complex(c, d))
        for a, b, c, d in [
            tuple(map(int, match.groups()))
            for match in  SENSOR_PATTERN.finditer(sensor_input.strip())
        ]
    ]

def beacon_free_columns(row, sensors):
    combined_zone = set()
    for sensor in sensors:
        combined_zone |= sensor.impossible_in_row(row)
    return combined_zone

def zone(sensor):
    return {sensor.sensor + complex(x, y)
     for y in range(-sensor.distance, sensor.distance + 1)
     for x in range(-(abs(sensor.distance)-abs(y)), abs(sensor.distance)-abs(y) + 1)
     if sensor.sensor + complex(x, y) not in (sensor.sensor, sensor.beacon) 
    }

def manhattan(a, b):
    distance = a - b
    return int(abs(distance.real) + abs(distance.imag))

def posible_beacons_in_space(sensors, xs, ys):
    possible = set()
    for row in range(ys[0], ys[1]):
        skips = sorted([sensor.skip_on_row(row) for sensor in sensors if sensor.on_row(row)], reverse=True)
        x, row_last = xs
        while skips:
            if x > row_last:
                break
            skip_first, skip_last = skips.pop()
            if skip_last < x:
                continue
            elif skip_first <= x:
                x = max(x, skip_last)
            else:
                possible.add(complex(x, row))
                x += 1
    return list(possible)

def tuning_frequency(position):
    return int(position.real * 4000000 + position.imag)

def print_sensors(sensors, xs=(-10, 20), ys=(-10, 20)):
    zones = set()
    for sensor in sensors:
        zones |= zone(sensor)

    sensor_positions = [sensor.sensor_position() for sensor in sensors]
    beacon_positions = [sensor.beacon_position() for sensor in sensors]

    for y in range(ys[0], ys[1]+1):
        row = []
        for x in range(xs[0], xs[1]+1):
            if (x, y) in sensor_positions:
                row.append("S")
            elif (x, y) in beacon_positions:
                row.append("B")
            else:
                row.append("#" if complex(x, y) in zones else ".")
        print(f"{y:02}" + "".join(row))
            
    
assert manhattan(complex(0, 0), complex(1, 2)) == 3
assert manhattan(complex(-2, -2), complex(2, 2)) == 8
assert manhattan(complex(42, -47), complex(42, -47)) == 0
assert manhattan(complex(1, 2), complex(3, 4)) == 4
assert (
    manhattan(complex(5234, -2345), complex(82, 5324)) ==
    manhattan(complex(82, 5324), complex(5234, -2345))
)

assert zone(Sensor(complex(0, 0), complex(1, 1))) == {
    complex(x, y)
    for x, y in (
        (-2, 0),
        (-1, 1), (-1, 0), (-1, -1),
        (0, 2), (0, 1), (0, -1), (0, -2),
        (1, 0), (1, -1),
        (2, 0)
    )
}

In [4]:
# Part 1 - test
sensors = parse_input(test_input)
assert len(beacon_free_columns(10, sensors)) == 26

In [5]:
# Part 1
sensors_and_beacons = parse_input(Path("input.txt").read_text())
len(beacon_free_columns(2000000, sensors_and_beacons))

5112034

In [6]:
# Part 2 - test
sensors = parse_input(test_input)
beacons = posible_beacons_in_space(sensors, xs=(0, 20), ys=(0, 20))
assert len(beacons) == 1
assert tuning_frequency(beacons[0]) == 56000011

In [7]:
# Part 2
sensors = parse_input(Path("input.txt").read_text())
beacons = posible_beacons_in_space(sensors, xs=(0, 4000000), ys=(0, 4000000))
assert len(beacons) == 1
tuning_frequency(beacons[0])

13172087230812